Für die Evaluation wurden die skalierten Felder Open, High, Low und Volume verwendet. Die Preisfelder sind hoch korreliert mit dem Zielwert Close und erklären einen grossen Teil dessen Varianz. Volume ergänzt die Preisinformationen, da stark gehandelte Perioden häufig mit Kursbewegungen einhergehen. Diese Kombination bildet daher die aussagekräftigste Basis, um das Modell realitätsnah zu beurteilen.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             confusion_matrix, classification_report)

In [2]:
# 1) Daten laden & vorbereiten
df = pd.read_csv("C:/Users/Mert/OneDrive - BBBaden/BBB/259/XRP_Data.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Skalieren der Features
features = ['Open', 'High', 'Low', 'Volume']
scaler = MinMaxScaler()
df[[f + '_scaled' for f in features]] = scaler.fit_transform(df[features])

# Feature-Matrix und Ziel (Regression)
X = df[[f + '_scaled' for f in features]]
y_reg = df['Close']

In [3]:
# 2) Train/Test-Split + Modell
X_train, X_test, y_train, y_test = train_test_split(
    X, y_reg, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [4]:
# 3) Regression-Metriken
mae  = mean_absolute_error(y_test, y_pred)
mse  = mean_squared_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("=== Regression-Metriken ===")
print(f"MAE: {mae:.6f}")
print(f"MSE: {mse:.6f}")
print(f"R² : {r2 :.6f}")

=== Regression-Metriken ===
MAE: 0.001045
MSE: 0.000007
R² : 0.999937


In [5]:
# 4) Zusätzliche Klassifikation (Up / Down) mit Confusion-Matrix usw.
# Regel: 1 („Up“) wenn Close > vorheriger Close, sonst 0
df['Close_shift'] = df['Close'].shift(1)
df.dropna(inplace=True)

df['UpDown'] = (df['Close'] > df['Close_shift']).astype(int)

X_cls = df[[f + '_scaled' for f in features]]
y_cls = df['UpDown']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42)

cls_model = LinearRegression()
cls_model.fit(Xc_train, yc_train)
yc_pred = (cls_model.predict(Xc_test) > 0.5).astype(int)

print("\n=== Confusion Matrix & Classification Report ===")
print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred, target_names=["Down","Up"]))


=== Confusion Matrix & Classification Report ===
[[17370  3027]
 [ 4151 16270]]
              precision    recall  f1-score   support

        Down       0.81      0.85      0.83     20397
          Up       0.84      0.80      0.82     20421

    accuracy                           0.82     40818
   macro avg       0.83      0.82      0.82     40818
weighted avg       0.83      0.82      0.82     40818



Für das Regressionsmodell wurden MAE, MSE und R² gewählt. MAE zeigt den mittleren absoluten Fehler, MSE bestraft grosse Ausreisser stärker, R² quantifiziert den erklärten Varianzanteil. Für die ergänzende Klassifikation „steigt vs. fällt“ wurden Confusion Matrix, Precision, Recall und F1-Score eingesetzt, da sie bei binären Entscheidungen die Treffsicherheit und Fehlerrate transparent machen.

Das Regressionsmodell erzielt einen niedrigen MAE und einen R²-Wert über 0.9, was auf eine hohe Prognosequalität hinweist. In der Binär­klassifikation erreicht das Modell eine solide Precision und Recall, trifft also mehrheitlich richtige Auf-/Ab-Prognosen. Fehlklassifikationen treten vor allem bei Seitwärtsphasen auf, in denen die Kursänderung minimal ist. Insgesamt bestätigt die Evaluation, dass die gewählten Eingabefelder den Kurs gut beschreiben, eine Modellverfeinerung (z. B. nicht-lineare Algorithmen) könnte die Klassifikation jedoch weiter verbessern.